In [ ]:
"""
Main entry point for the JOULE process calculator.

This module provides the entry point for running the JOULE process calculator 
as both a command-line application and a Jupyter notebook interface.
"""
import os
import sys
import argparse

from models import gas_properties
from models.joule_process import JouleProcessCalculator
from utils import converters
from utils.converters import bar_to_pascal  # celsius_to_kelvin nicht mehr importieren
from visualization import results_formatter  
from visualization.results_formatter import print_results_table, material_properties_table
from visualization.plotting import plot_process  
from ui.widgets import run_joule_calculator


def main():
    """
    Command-line interface for the JOULE process calculator
    """
    parser = argparse.ArgumentParser(description='JOULE-Prozess Rechner')
    
    # Gas selection
    parser.add_argument('--gas', type=str, default='air',
                        choices=['air', 'helium', 'nitrogen', 'carbon_dioxide'],
                        help='Working fluid (default: air)')
    
    # Cp model
    parser.add_argument('--cp-model', type=str, default='constant',
                        choices=['constant', 'temperature_dependent'],
                        help='Cp model (default: constant)')
    
    # Regeneration
    parser.add_argument('--regeneration', action='store_true',
                        help='Enable regeneration')
    parser.add_argument('--reg-eff', type=float, default=0.8,
                        help='Regeneration efficiency (default: 0.8)')
    
    # Component efficiencies
    parser.add_argument('--comp-eff', type=float, default=0.85,
                        help='Compressor isentropic efficiency (default: 0.85)')
    parser.add_argument('--turb-eff', type=float, default=0.9,
                        help='Turbine isentropic efficiency (default: 0.9)')
    
    # Mass flow
    parser.add_argument('--mass-flow', type=float, default=None,
                        help='Mass flow in kg/s (default: None)')
    
    # State 1
    parser.add_argument('--p1', type=float, default=1.0,
                        help='Pressure p1 in bar (default: 1.0)')
    parser.add_argument('--T1', type=float, default=293.15,  # Geändert auf Kelvin
                        help='Temperature T1 in K (default: 293.15)')  # Geändert
    
    # State 2
    parser.add_argument('--p2', type=float, default=8.0,
                        help='Pressure p2 in bar (default: 8.0)')
    
    # State 3
    parser.add_argument('--T3', type=float, default=1273.15,  # Geändert auf Kelvin
                        help='Temperature T3 in K (default: 1273.15)')  # Geändert
    
    # State 4
    parser.add_argument('--p4', type=float, default=1.0,
                        help='Pressure p4 in bar (default: 1.0)')
    
    # Output options
    parser.add_argument('--plot', action='store_true',
                        help='Generate plots')
    parser.add_argument('--diagram', type=str, default='Ts',
                        choices=['Ts', 'pv', 'hs'],
                        help='Diagram type (default: Ts)')
    parser.add_argument('--save-plot', type=str, default=None,
                        help='Save plot to file (provide filename)')
    parser.add_argument('--show-material-props', action='store_true',
                        help='Show material properties')
    
    args = parser.parse_args()
    
    # Create calculator instance
    calc = JouleProcessCalculator(
        gas=args.gas, 
        use_const_cp=(args.cp_model == 'constant')
    )
    
    # Calculate states
    p1 = bar_to_pascal(args.p1)
    T1 = args.T1  # Keine Umrechnung mehr nötig
    calc.calculate_state_1(p1, T1)
    
    p2 = bar_to_pascal(args.p2)
    calc.calculate_state_2(p2)
    
    p3 = p2  # Isobar
    T3 = args.T3  # Keine Umrechnung mehr nötig
    calc.calculate_state_3(p3, T3)
    
    p4 = bar_to_pascal(args.p4)
    calc.calculate_state_4(p4)
    
    # Set process parameters
    calc.set_parameters(
        regeneration=args.regeneration,
        reg_eff=args.reg_eff if args.regeneration else 0.0,
        compressor_efficiency=args.comp_eff,
        turbine_efficiency=args.turb_eff,
        mass_flow=args.mass_flow
    )
    
    # Suggested code change: assign T1, T2, T3, T4 as floats from states
    T1 = float(calc.states[1]["T"])
    T2 = float(calc.states[2]["T"])
    T3 = float(calc.states[3]["T"])
    T4 = float(calc.states[4]["T"])

    print(f"T2 = {calc.states[2]['T']} (type: {type(calc.states[2]['T'])})")
    print(f"T4 = {calc.states[4]['T']} (type: {type(calc.states[4]['T'])})")

    if T4 <= T2:
        # ...
        pass

    # In calculate_state_4
    #dT_isentrop = T3 - T4s  # Diese Differenz ist positiv
    #dT_real = dT_isentrop * self.turbine_efficiency
    #T4 = T3 - dT_real

    # Calculate process properties
    props = calc.calculate_process_properties()
    
    # Calculate optimal pressure ratio
    pi_opt, T2_opt = calc.calculate_optimal_pressure_ratio()
    
    # Print results
    print("JOULE-Prozess Rechner")
    print(f"Optimales Druckverhältnis: π_opt = {pi_opt:.4f}")
    
    # Use IPython display if available, otherwise print standard output
    try:
        from IPython.display import display
        in_notebook = True
    except ImportError:
        in_notebook = False
    
    if in_notebook:
        print_results_table(calc)
        
        if args.show_material_props:
            material_properties_table(calc)
        
        if args.plot:
            fig, ax = plot_process(calc, diagram_type=args.diagram, save_fig=bool(args.save_plot), filename=args.save_plot)
    else:
        # Print state variables in a simple table format
        print("\nZustandsgrößen:")
        print("---------------")
        for i in sorted(calc.states.keys()):
            state = calc.states[i]
            print(f"Zustand {i}:")
            print(f"  p = {state['p']/1e5:.4f} bar")
            print(f"  T = {state['T']:.2f} K = {state['T']-273.15:.2f} °C")
            print(f"  v = {state['v']:.6f} m³/kg")
            print(f"  h = {state['h']:.2f} J/kg")
            print(f"  s = {state['s']:.4f} J/(kg·K)")
        
        # Print process properties
        print("\nProzessgrößen:")
        print("-------------")
        if 'eta_th' in props:
            print(f"η_th = {props['eta_th']:.4f}")
        print(f"w_KP = {props['w_kp']:.2f} J/kg")
        print(f"q_zu = {props['q_in']:.2f} J/kg")
        if 'q_out' in props:
            print(f"q_ab = {props['q_out']:.2f} J/kg")
        if 'pi' in props:
            print(f"π = {props['pi']:.4f}")
        if 'tau' in props:
            print(f"τ = {props['tau']:.4f}")
        
        # Print power values if mass flow is provided
        if args.mass_flow is not None and all(k in props for k in ['P_comp', 'P_turb', 'P_kp', 'Q_in', 'Q_out']):
            print("\nLeistungen:")
            print("----------")
            print(f"P_comp = {props['P_comp']/1000:.2f} kW")
            print(f"P_turb = {props['P_turb']/1000:.2f} kW")
            print(f"P_KP = {props['P_kp']/1000:.2f} kW")
            print(f"Q_zu = {props['Q_in']/1000:.2f} kW")
            print(f"Q_ab = {props['Q_out']/1000:.2f} kW")
            if args.regeneration and 'Q_reg' in props:
                print(f"Q_reg = {props['Q_reg']/1000:.2f} kW")
        
        # Generate plot if requested
        if args.plot:
            fig, ax = plot_process(calc, diagram_type=args.diagram, save_fig=bool(args.save_plot), filename=args.save_plot)
            import matplotlib.pyplot as plt
            plt.show()


def run_notebook():
    """
    Run the JOULE process calculator in Jupyter notebook mode
    """
    run_joule_calculator()


if __name__ == "__main__":
    if 'ipykernel' in sys.modules:
        # Running in a Jupyter notebook
        run_notebook()
    else:
        # Running as a script
        main()